In [1]:
def read_names(file_path):
    """Read city names from a file."""
    with open(file_path, 'r') as file:
        names = file.read().splitlines()
    return names

def read_distances(file_path):
    """Read city distance matrix from a file."""
    with open(file_path, 'r') as file:
        distances = [[float(value) for value in line.split()] for line in file]
    return distances

def calculate_tour_distance(tour, distances):
    """Calculate the total distance of a tour."""
    total_distance = 0
    for i in range(len(tour) - 1):
        total_distance += distances[tour[i]][tour[i + 1]]
    total_distance += distances[tour[-1]][tour[0]]  # Return to start
    return total_distance

In [2]:
# Greedy Algorithm
def find_nearest_city(last_city, unvisited, distances):
    """Find the nearest unvisited city to the current city."""
    nearest_city = None
    nearest_city_distance = float('inf')
    for candidate_city in unvisited:
        distance = distances[last_city][candidate_city]
        if distance < nearest_city_distance:
            nearest_city_distance = distance
            nearest_city = candidate_city
    return nearest_city

def tsp_greedy(name_file, distance_file):
    """
    This function takes as input the paths to two files: one containing the names of the cities,
    and the other containing the distances between each pair of cities. It then computes the
    shortest possible tour that visits each city exactly once and returns to the starting city,
    by following a greedy approach. For each city used as a starting point, the algorithm
    constructs a tour that at each step extends the current tour by moving to the nearest
    unvisited city. The function finally prints the names of the cities in the order they are
    visited in the best tour found, as well as the total distance of this tour.
    Parameters:
    - names (str): The file path to the text file containing city names.
    - distances (str): The file path to the text file containing the distance matrix,
      with each row representing the distances from a city to all other cities.
    """
    names = read_names(name_file)
    distances = read_distances(distance_file)
    best_tour = []
    best_distance = float('inf')

    for start_city in range(len(names)):
        current_tour = [start_city]
        unvisited = set(range(len(names))) - {start_city}

        while unvisited:
            last_city = current_tour[-1]
            nearest_city = find_nearest_city(last_city, unvisited, distances)
            current_tour.append(nearest_city)
            unvisited.remove(nearest_city)

        current_distance = calculate_tour_distance(current_tour, distances)
        if current_distance < best_distance:
            best_distance = current_distance
            best_tour = current_tour + [current_tour[0]]
    best_tour_names = []
    for city in best_tour:
        best_tour_names.append(names[city])
        
    print("Best Tour:", best_tour_names)
    print("Total Distance:", best_distance)
tsp_greedy('thirty_cities_names.txt', 'thirty_cities_dist.txt')

Best Tour: ['Azores', 'London', 'Paris', 'Berlin', 'Rome', 'Istanbul', 'Cairo', 'Baghdad', 'Moscow', 'Bombay', 'Shanghai', 'Tokyo', 'Guam', 'Manila', 'Melbourne', 'Sydney', 'Honolulu', 'San Francisco', 'Seattle', 'Juneau', 'Chicago', 'New York', 'Montreal', 'New Orleans', 'Mexico City', 'Panama City', 'Santiago', 'Buenos Aires', 'Rio de Janeiro', 'Capetown', 'Azores']
Total Distance: 527.0


In [3]:
import random
def mutate_tour(tour):
    """Make a small random change in the TSP solution to generate another solution."""
    tour_copy = tour[:]
    city1, city2 = random.sample(range(len(tour)), 2)
    tour_copy[city1], tour_copy[city2] = tour_copy[city2], tour_copy[city1]
    return tour_copy

def tsp_evolution(name_file, distance_file):
    """Perform the evolutionary algorithm to find a TSP solution."""
    global best_solution, best_distance
    names = read_names(name_file)
    distances = read_distances(distance_file)
    current_solution = list(range(len(names)))  # Initial TSP solution
    current_distance = calculate_tour_distance(current_solution, distances)
    best_solution = current_solution[:]
    best_distance = current_distance
    stagnations = 0

    while stagnations < 5:
        new_solution = mutate_tour(current_solution)
        new_distance = calculate_tour_distance(new_solution, distances)

        if new_distance < best_distance:
            best_solution = new_solution
            best_distance = new_distance
            stagnations = 0  # Reset stagnations
        else:
            stagnations += 1  # Increment stagnation counter

        if stagnations == 5:
            better_solution_found = False
            # Mutate three times to escape local optimum
            for _ in range(3):
                current_solution = mutate_tour(current_solution)
                current_distance = calculate_tour_distance(current_solution, distances)
                if current_distance < best_distance:
                    best_solution = current_solution
                    best_distance = current_distance
                    better_solution_found = True
                    break  # Exit the loop if a better solution is found
            if better_solution_found:
                stagnations = 0  # Reset stagnations if a better solution is found
            else:
                stagnations = 5  # Maintain stagnation count to force termination
    best_tour_names = []
    for city in best_solution:
        best_tour_names.append(names[city])
    best_tour_names.append(names[best_solution[0]])
    print("Best Tour:", best_tour_names)
    print("Total Distance:", round(best_distance, 2))
tsp_evolution('thirty_cities_names.txt', 'thirty_cities_dist.txt')


Best Tour: ['Azores', 'Baghdad', 'Berlin', 'Bombay', 'Buenos Aires', 'Cairo', 'Capetown', 'Chicago', 'Guam', 'Honolulu', 'Istanbul', 'Juneau', 'London', 'Manila', 'Melbourne', 'Mexico City', 'Montreal', 'Moscow', 'New Orleans', 'New York', 'Panama City', 'Paris', 'Rio de Janeiro', 'Rome', 'San Francisco', 'Santiago', 'Seattle', 'Shanghai', 'Sydney', 'Tokyo', 'Azores']
Total Distance: 1616.0


In [4]:
# Backtracking Algorithm
# Global variables to track the best tour and its distance
best_tour = None
best_distance = float('inf')

def tsp_recursion(partial_tour, remaining_cities, distances):
    """
    Recursively search for the shortest TSP tour using backtracking.
    This function updates the global best_tour and best_distance variables with the best
    tour found during the recursion.
    Parameters:
    - partial_tour (list of int): The current partial tour being constructed.
    - remaining_cities (list of int): Indices of cities that have not been visited yet.
    - distances (list of list of float): A 2D list (matrix) of distances between cities.
    """
    global best_tour, best_distance
    if not remaining_cities:
        partial_tour.append(partial_tour[0])  # Complete the tour
        current_distance = calculate_tour_distance(partial_tour, distances)
        if current_distance < best_distance:
            best_distance = current_distance
            best_tour = partial_tour[:]
        partial_tour.pop() 
        return
    
    for next_city in remaining_cities:
        if not partial_tour or next_city > partial_tour[-1]:  # Avoid flips and redundant solutions
            new_remaining_cities = remaining_cities.copy()
            new_remaining_cities.remove(next_city)
            tsp_recursion(partial_tour + [next_city], new_remaining_cities, distances)

def tsp_backtrack(name_file, distance_file):
    """
    Find the optimal TSP tour and its total distance using backtracking.
    Reads city names and distances from files, initializes the backtracking search,
    and prints the best tour and its total distance.
    Parameters:
    - name_file (str): The file path to the text file containing city names.
    - distance_file (str): The file path to the text file containing the distance matrix.
    """
    global best_tour, best_distance
    cities = read_names(name_file)  # Use the updated function name
    distances = read_distances(distance_file)
    start_city = 0 
    tsp_recursion([start_city], list(range(1, len(cities))), distances)
    best_tour_names = [cities[index] for index in best_tour]
    print("Best Tour:", best_tour_names)
    print("Total Distance:", best_distance)

tsp_backtrack('seven_cities_names.txt', 'seven_cities_dist.txt')


Best Tour: ['Alpha', 'Beta', 'Gamma', 'Delta', 'Epsilon', 'Zeta', 'Eta', 'Alpha']
Total Distance: 157.4
